---
title: "Pile driveability"
author: "Kevin Duffy"
bibliography: ../bibliography.bib
draft: true
---

### Pre holidays 25/07/25
- Was in the middle of doing the script (pile-installation_NEWEST.py) and adding a hammer spring but getting a few stability issues. 

Pile driveability serves four key purposes:

- Determine whether driven installation is a feasible option in getting the piles to the target depth
- Assess the likelihood and mitigate of pile damage to excessive stresses
- Hammer selection
- Establish the resources needed for pile driving (e.g. time, cost, fuel, rigging equipment)

Software such as GRLWEAP or AllWave.

**Explain why driveability methods are more common offshore**

## Basic concepts
(Wave equation)

To simulate the shaft friction in a driveability study, we use a model quite similar to what was defined in **CHAPTER LOAD-TRANSFER**. pile-soil interface, we use what's called a **spring-damper** model. 

Pile driving is a dynamic process, and so energy itself is transferred from pile-to-soil, which is turned dampened (in other words, energy is lost) by molecular interactions in the soil itself. To account for this energy loss using a viscous damper.

![Simulating pile-soil springs](pics/installation/analytical-model.png){#fig-install-model width=60%}


## Pile-soil interaction
We need to define how the pile interacts with the soil to "complete" our wave equation model. The core of most driveability assessments and software is based on the rheological framework proposed by @smithPiledrivingAnalysisWave1960. @smithPiledrivingAnalysisWave1960 proposed that the pile displacement $w$ and velocity $v$ as a function of a spring, dashpot and plastic slider (**FIGURE REF**) whereby:

$$ 
T = T_s + T_d = Kw + Cv
$$

* The spring stiffness $K$ models recoverable static resistance, subject to $T_s \le T_{max}$ (i.e. the limit of the plastic slider)
* The plastic slider represents the yield strength of the soil $T_{max}$, essentially the point at which the pile "slips" through and no further resistance can be deleloped. The deformation at which $T_{max}$ is referred to as the **soil quake**
* The viscous dashpot $C$ adds a rate-dependency into the model, and essentially dampens the movement of the spring. 

With that said, this is a relatively simple formulation. It misse, and so this modern forms of this equation are typically taken as:

$$ 
T = \min (1,\frac{w}{Q})(1+Jv^n)T_{max}
$$

where, generally, the velocity power n is taken as unity, so that the dimensions of the ‘Smith’ damping coefficient J are the inverse of velocity, rather than the dimensions of C in equation (27) which are force per velocity. The quantity, Q, is the ‘quake’ or displacement at which the ultimate  static resistance, Tmax, is mobilised.



You're probably unsurprised to hear that there's still shortcoming to this! 
See the IMPACT manual

#### Quake
Shaft quakes vary relatively little, with a quake of 2.5 mm generally considered reasonable for most soils. However, toe quakes can vary widely, as the value is highly dependent on the soil stiffness around the pile base (**REF REF**)--something which is dependent on the installation procedure and the soil type, as well as an interdependency between the two (**REF SECTION**)

#### Damping 
How damping is accounted for can vary quite a bit. As a basic model, GRLWEAP takes

$$ 
R_{di} = j_{si} | R_{si} | v_i
$$

where $R_{di}$ is  the damping resistance force and $j_{si}$, $v_i$ and $R_{si}$  are the Smith damping factor, the pile segment velocity, and the current static resistance force, all at segment $i$, respectively. Smith's damping factor has units of time over length. Although the Smith model was developed over half a century ago, the same basic factors for damping along the shaft, i.e., 0.15 s/m (0.05 s/ft) for sands and 0.65 s/m (0.2 s/ft) for clays are still recommended. Different values are used where measurements are made or in mixed soils. Only the toe damping recommendation changed to 0.5 s/m (0.15 s/ft) for all soil types

## Pile hammer

## Static resistance to driving (SRD) methods
An SRD method is very similar to the static capacity methods we looked at (REF, REF) since it more or less aims to capture the same interaction mechanisms. However there are some key differences:

* Static capacity methods include some degree of aging, whereas SRD methods give the instanteous capacity at the end of driving
* Influence of the soil plug is very different under dynamic loading compared to static loading
* SRD methods generally use best or upper-bound estimates of resistance, instead of lower bound estimates. This is to ensure that the chosen hammer can be used for all piles at a given site, reducing the likelihood of having to return onshore to get another hammer
* Pore pressure development under high strains

## Monitoring


In [1]:
# Re-import necessary packages after kernel reset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# --- Time setup ---
dt = 0.05
t = np.arange(0, 10, dt)
w = 0.03 * np.sin(2 * np.pi * 0.8 * t)  # pile head displacement
v = np.gradient(w, dt)

# --- Soil layer parameters ---
# Layer 1 (upper shaft layer)
Tmax1 = 10000  # N
Q1 = 0.005     # m
J1 = 150       # s/m

# Layer 2 (lower shaft layer)
Tmax2 = 20000  # N
Q2 = 0.01      # m
J2 = 300       # s/m

# Base resistance
Tmax_base = 50000  # N
Qb = 0.007         # m
Jb = 100           # s/m

# --- Compute resistances using modern formulation ---
def modern_resistance(w, v, Tmax, Q, J):
    stat = np.minimum(1, np.abs(w)/Q) * Tmax * np.sign(w)
    dyn = J * np.abs(v) * Tmax * np.sign(v)
    return (np.minimum(1, np.abs(w)/Q) * (1 + J * np.abs(v))) * Tmax * np.sign(w)

T1 = modern_resistance(w, v, Tmax1, Q1, J1)
T2 = modern_resistance(w, v, Tmax2, Q2, J2)
Tb = modern_resistance(w, v, Tmax_base, Qb, Jb)

# --- Plot setup ---
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(0, t[-1])
ax.set_ylim(-80000, 80000)
ax.set_xlabel("Time (s)")
ax.set_ylabel("Resistance (N)")
ax.set_title("Modern Soil Resistance: Shaft and Base (Two Layers)")

line_T1, = ax.plot([], [], 'b-', label="Shaft Resistance (Layer 1)")
line_T2, = ax.plot([], [], 'g-', label="Shaft Resistance (Layer 2)")
line_Tb, = ax.plot([], [], 'r--', label="Base Resistance")
line_Tsum, = ax.plot([], [], 'k-', lw=2, label="Total Resistance")

ax.legend(loc='upper right')

def init():
    line_T1.set_data([], [])
    line_T2.set_data([], [])
    line_Tb.set_data([], [])
    line_Tsum.set_data([], [])
    return line_T1, line_T2, line_Tb, line_Tsum

def update(i):
    ti = t[:i]
    Tsum = T1[:i] + T2[:i] + Tb[:i]
    line_T1.set_data(ti, T1[:i])
    line_T2.set_data(ti, T2[:i])
    line_Tb.set_data(ti, Tb[:i])
    line_Tsum.set_data(ti, Tsum)
    return line_T1, line_T2, line_Tb, line_Tsum

ani = animation.FuncAnimation(fig, update, frames=len(t), init_func=init,
                              interval=50, blit=True)
plt.close(fig)
ani

a

## Pile run

### Intermediate soils

### Slight mention of carbonates